# Основные операторы PostgreSQL (vo_HW)

### Журавлев А.Н.

## 1. (1 балл) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

**Решение.**       

У нас 2 проблемы:
- атрибут *standard_cost* имеет тип данных `varchar`
- в этой же колонке имеются пустые значения в виде `''`

In [ ]:
-- заменяем '' на NULL в столбце standard_cost
UPDATE transaction
SET standard_cost = NULL
	WHERE standard_cost = '';

In [ ]:
-- переводим значения атрибута standard_cost из varchar в float
ALTER TABLE transaction
ALTER COLUMN standard_cost TYPE FLOAT
USING CAST(REPLACE(standard_cost, ',', '.') AS FLOAT);

А теперь выводим уникальные бренды, у которых стандартная соимость выше 1500

In [ ]:
select 
	distinct(t.brand)
from "transaction" as t
where t.standard_cost  > 1500;

![](imgs/image_01.png)

## 2. (1 балл) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.

**Решение.**

Атрибут *transaction_date* имеет тип данных `varchar`. Дата указана в виде `dd.mm.yyyy`    

Проверим на наличие проблем:

In [ ]:
SELECT transaction_date
FROM transaction
WHERE transaction_date IS NULL OR transaction_date !~ '^\d{2}\.\d{2}\.\d{4}$';

Проблем нет.

Теперь преобразуем данные:

In [ ]:
ALTER TABLE transactions
ALTER COLUMN transaction_date TYPE DATE
USING TO_DATE(transaction_date, 'DD.MM.YYYY');

Подтвержденные транзакции - информация об этом, на сколько я понял, в атрибуте *order_status*.      
Теперь глянем какой статус у транзакций бывает:

In [ ]:
select distinct (order_status)
from transaction;

Получается 2 статуса: *Approved* и *Cancelled*

Теперь выводим все подтвержденные транзакции за указанный период включительно (чтобы проверить себя отсортируем данные по дате по возрастанию):

In [ ]:
SELECT *
FROM transaction
WHERE (transaction_date BETWEEN '2017-04-01' AND '2017-04-09') and order_status = 'Approved'
order by transaction_date;

![](imgs/image_02.png)

## 3. (1 балл) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

В задании явно не указано, чтобы я вывел уникальные значения профессий, тогда код будет следующим:

In [ ]:
select 
	job_title
from customer
where 
	(job_industry_category = 'IT' or job_industry_category = 'Financial Services') and
	job_title like 'Senior%';

![](imgs/image_03_1.png)

Всего 81 запись. 

Однако, думаю, будет правильным вывести уникальные профессии по заданным условиям:

In [ ]:
select 
	distinct(job_title)
from customer
where 
	(job_industry_category = 'IT' or job_industry_category = 'Financial Services') and
	job_title like 'Senior%';


![](imgs/image_03_2.png)

## 4. (1 балл) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services

Пришло время объединять таблицы.     
Основная таблица, в контексте нашего запроса, `transaction`, т.к. именно по ней мы будем знать какие бренды покупают клиенты. Следовательно, используем `LEFT JOIN` для присоединения к ней таблицы `customer`.   
   
Все-таки, я думаю, что в задании имеется в виду вывести уникальные бренды:

In [ ]:
select 
	distinct(brand)
from transaction t
left join customer c on t.customer_id = c.customer_id
where 
	job_industry_category = 'Financial Services';

![](imgs/image_04_1.png)

Все неплохо, но бренд под номером 1 - пустой. Может я ошибся при объединении таблиц?    

Проверим на исходной таблице `transaction`:

In [ ]:
select distinct(brand)
from transaction

![](imgs/image_04_2.png)

Нет, ошибки нет, в исходной таблице `transaction` также есть пустой бренд.

## 5. (1 балл) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

In [ ]:
select 
	first_name,
	last_name
from transaction t
left join customer c on t.customer_id = c.customer_id
where 
	online_order and
	brand in ( 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
limit 10;

Либо, если в задании имеются в виду уникальные клиенты:

In [ ]:
select distinct
	first_name,
	last_name
from transaction t
left join customer c on t.customer_id = c.customer_id
where 
	online_order and
	brand in ( 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
limit 10;

![](imgs/image_05.png)

![](imgs/image_05.png)

## 6. (1 балл) Вывести всех клиентов, у которых нет транзакций.

В основно таблице `transaction` каждая транзакция должна иметь *transaction_id* и *customer_id*.     
На всякий случай проверим, чтобы эти поля не имели пропусков:

In [ ]:
select *
from transaction
where customer_id is null or transaction_id is null;

Пропусков нет, все в порядке.

Чтобы вывести клиентов, у которых нет транзакций, надо к основной таблице `transaction` через `RIGHT JOIN` присоединить таблицу `customer` (либо наоборот, но уже через `LEFT JOIN`), тогда в общей таблице у клиентов без транзакций значение поля `transaction_id` будет пустым (либо, например, значение поля `customer_id` таблицы `transaction` тоже будет пустым). Вот один из вариантов решения:

In [ ]:
select distinct
	t.customer_id,
	t.transaction_id transaction_id, 
	first_name,
	last_name
from transaction t
right join customer c on t.customer_id = c.customer_id
where transaction_id is null or t.customer_id is null;

![](imgs/image_06.png)

![](imgs/image_06.png)

Всего 507 человек (без `distinct` ничего не изменится). 

## 7. (2 балла) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

In [ ]:
select distinct
    t.customer_id,
	first_name,
	last_name
from transaction t
left join customer c on t.customer_id = c.customer_id
where 
	job_industry_category = 'IT' and
	t.standard_cost = (
      SELECT MAX(standard_cost)
      FROM transaction
  );

![](imgs/image_07.png)

![](imgs/image_07.png)

## 8. (2 балла) Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

In [ ]:
select distinct
	t.customer_id,
	first_name,
	last_name
from transaction t
left join customer c on t.customer_id = c.customer_id
where 
	job_industry_category in ('IT', 'Health') and
	(transaction_date BETWEEN '2017-07-07' AND '2017-07-17') and 
	order_status = 'Approved';

![](imgs/image_08.png)

![](imgs/image_08.png)

Всего 115 уникальных клиентов.